In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os, sys, time
%matplotlib inline

In [2]:
import torch as th
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable 


In [30]:
def hash(z):
    h = 0
    for e in z:
        h = 10*h + e 
    return h

class MyNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs))
        y_logits = self.fc2(x)
        return y_logits

def print_eval(model, X, y, n_epochs):
    print('======================================')
    print('{} epochs have been finished'.format(n_epochs))    
    logits = model(X)
    
    targets = y.data.numpy()
    _, preds = F.softmax(logits).data.max(1)
    preds = preds.numpy()
    
    print('Loss:', cross_entropy(F.log_softmax(logits), y).data[0], end=' ')
    print('accuracy:', (targets == preds).mean())
    print('Targets:', targets)
    print('Preds:  ', preds)


input_dim = 10
H = 20
output_dim = 4

N = 10

X = (th.randn(N,input_dim)*5).round().div(5)
X = Variable(X)
y = Variable(th.LongTensor(N).random_(output_dim)) #requires_grad is False by default 

print('inputs')
print(X)
print('targets')
print(y)

model = MyNetwork(input_dim, H, output_dim)
initial_params = [(name, p.data.clone()) for name, p in model.named_parameters()]
first_layer_params = [p for name, p in model.named_parameters() if 'fc1' in name]


cross_entropy = nn.NLLLoss()
optimizer = optim.SGD(first_layer_params, lr=1e-3, momentum=0.9)

n_steps = 500
n_epochs = 6 
for epoch in range(n_epochs):
    print_eval(model, X, y, epoch)
    for i in range(n_steps):
        log_preds = F.log_softmax(model(X))
        loss = cross_entropy(log_preds, y)
        model.zero_grad()
        
        loss.backward()
        optimizer.step()

print_eval(model, X, y, n_epochs)

inputs
Variable containing:
-0.2000 -0.0000 -1.0000  1.2000  0.6000 -1.2000  0.0000  0.4000  1.4000  0.4000
 0.4000  1.0000 -2.2000 -0.0000  0.2000 -0.4000 -0.4000  1.4000  0.8000 -2.2000
-1.4000 -1.0000 -0.2000 -1.0000  0.2000 -0.8000 -0.2000 -0.6000 -1.4000  0.6000
-0.4000 -2.0000  0.4000  0.2000 -1.6000 -1.0000  0.4000  1.2000 -1.8000 -1.0000
 0.2000  0.6000  1.4000  2.0000 -0.2000 -0.2000  1.6000 -1.2000  0.2000  0.8000
 1.2000 -0.8000  0.8000  0.2000 -0.4000 -0.6000 -0.4000 -1.0000 -1.8000 -0.2000
 0.0000 -1.2000 -0.4000 -0.6000 -1.0000  0.2000 -0.8000  1.0000 -1.4000  0.2000
 1.0000  1.8000  1.0000  1.6000  0.6000 -0.8000 -0.4000  0.4000  0.4000  1.2000
-0.0000  1.8000 -0.6000 -1.6000 -2.0000 -0.6000  0.4000 -0.8000 -0.2000 -2.0000
-1.0000  1.4000  1.6000  0.0000  0.2000 -1.0000 -0.2000 -0.2000  1.2000 -1.6000
[torch.FloatTensor of size 10x10]

targets
Variable containing:
 3
 2
 1
 0
 2
 2
 0
 0
 3
 3
[torch.LongTensor of size 10]

0 epochs have been finished
Loss: 1.43478262424

In [25]:
print('Check if params are the same:')
trained_params = list(model.named_parameters())

for i in range(len(trained_params)):
    name_init, p_init = initial_params[i]
    name_new, p_new = trained_params[i]
    assert name_init == name_new, 'Мы сравниваем только одинаковые параметры!'
    print('==========================')
    print(name_new, 'shapes:', p_init.size(), p_new.size())
    print('params are the same:', th.equal(p_init, p_new.data))

Check if params are the same:
fc1.weight shapes: torch.Size([20, 10]) torch.Size([20, 10])
params are the same: False
fc1.bias shapes: torch.Size([20]) torch.Size([20])
params are the same: False
fc2.weight shapes: torch.Size([4, 20]) torch.Size([4, 20])
params are the same: True
fc2.bias shapes: torch.Size([4]) torch.Size([4])
params are the same: True


In [4]:
foo = Variable(th.zeros(3,3))

foo1 = foo.detach()
foo1[0,0] = 1.
print('foo1')
print(foo1)
print('foo')
print(foo)

RuntimeError: in-place operations can be only used on variables that don't share storage with any other variables, but detected that there are 2 objects sharing it

In [ ]:
from IPython.display import clear_output


for i in range(40):
    print('step%d' % i)
    sys.stdout.flush()
    time.sleep(4.)
    clear_output(wait=True)

step33


In [17]:
import _pickle as pkl
from collections import namedtuple

TrainingStats = namedtuple('TrainingStats', "mean_r max_r min_r std_r targets_ratio preds_ration acc prec rec")
FooStats = namedtuple('FooStats', "epoch iter sum")

In [19]:
def test_dump(N, M, filename='test'):
    
    
    for epoch in range(N):
        l = []
        for t in range(M):
            l.append(FooStats(epoch, t, epoch + t))
            
        with open(filename +'.pickle', 'ba') as file:
            pkl.dump(l, file, protocol=4)
        
        
test_dump(2, 10)

In [56]:
def test_load(filename):
    data = []
    with open(filename, 'rb') as file:
        while True:
            try:
                data.extend(pkl.load(file))
            except EOFError:
                break
    return data

data = test_load('test.pickle')
for row in data:
    print(row)

FileNotFoundError: [Errno 2] No such file or directory: 'test.pickle'

In [108]:
def test_stat_saving(max_global_steps, checkpoint_interval, eval_every,
               num_emulators=32, max_local_steps=10, verbose=2, dump_fun=None):
    global_step = 0
    counter = 0
    last_saving_step = 0
    
    data = dict(num_eval=0, num_save=0)
    def eval_func(verbose=False):
        data['num_eval'] += 1
        if verbose > 1:
            print('#{0} EVALUATE: step={1} counter={2}'.format(data['num_eval'], global_step, counter))
        return TrainingStats(*[global_step + i for i in range(9)])
        
    def save_func(l, verbose=False):
        data['num_save'] += 1
        if verbose > 0:
            print('#{0} SAVE_CHECKPOINT: step={1} counter={2}'.format(data['num_save'], global_step, counter))
        if (dump_fun is not None) and len(l) > 0:
            dump_fun(l)
    l = []
    while global_step < max_global_steps:
        global_step += num_emulators * max_local_steps
        counter += 1
        
        if counter % (eval_every // (num_emulators * max_local_steps)) == 0:
            stats = eval_func(verbose)
            l.append(stats)
            
        if global_step - last_saving_step >= checkpoint_interval:
            last_saving_step = global_step
            save_func(l, verbose)
            l = []
            
    save_func(l)
    
import time, datetime
with open('test/test_pickle.p', 'ba') as file:            
    pickler = pkl.Pickler(file, protocol=4)
    #dump = None
    dump = pickler.dump
    #dump = lambda o: pkl.dump(o, file, protocol=4)
    
    start_time = time.time()
    test_stat_saving(max_global_steps=80*(10**6), checkpoint_interval=10**6,
                       eval_every=10240, verbose=1, dump_fun=dump)
    delta_t = time.time() - start_time
    print('seconds:', delta_t)
    print('time:', datetime.timedelta(seconds=delta_t))

#1 SAVE_CHECKPOINT: step=1000000 counter=3125
#2 SAVE_CHECKPOINT: step=2000000 counter=6250
#3 SAVE_CHECKPOINT: step=3000000 counter=9375
#4 SAVE_CHECKPOINT: step=4000000 counter=12500
#5 SAVE_CHECKPOINT: step=5000000 counter=15625
#6 SAVE_CHECKPOINT: step=6000000 counter=18750
#7 SAVE_CHECKPOINT: step=7000000 counter=21875
#8 SAVE_CHECKPOINT: step=8000000 counter=25000
#9 SAVE_CHECKPOINT: step=9000000 counter=28125
#10 SAVE_CHECKPOINT: step=10000000 counter=31250
#11 SAVE_CHECKPOINT: step=11000000 counter=34375
#12 SAVE_CHECKPOINT: step=12000000 counter=37500
#13 SAVE_CHECKPOINT: step=13000000 counter=40625
#14 SAVE_CHECKPOINT: step=14000000 counter=43750
#15 SAVE_CHECKPOINT: step=15000000 counter=46875
#16 SAVE_CHECKPOINT: step=16000000 counter=50000
#17 SAVE_CHECKPOINT: step=17000000 counter=53125
#18 SAVE_CHECKPOINT: step=18000000 counter=56250
#19 SAVE_CHECKPOINT: step=19000000 counter=59375
#20 SAVE_CHECKPOINT: step=20000000 counter=62500
#21 SAVE_CHECKPOINT: step=21000000 counte

In [109]:
from sys import getsizeof
def load_pickled(filename):
    with open(filename, 'br') as file:   
        unpickler = pkl.Unpickler(file)
        data = []
        while True:
            try:
                data.extend(unpickler.load())
            except EOFError as e:
                break
    return data
        
data = load_pickled('test/test_pickle.p')


In [116]:
mem_size = len(data)*9*getsizeof(2.4)


In [121]:
mem_size / (1024*1024) 


1.609222412109375

In [117]:
del data